In [2]:
# data cleaning
import numpy as np
import pandas as pd

# feature engineering
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans

# model
from xgboost import XGBRegressor

# error score
from sklearn.model_selection import cross_val_score

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# load data
data = pd.read_csv(r'train.csv', index_col='Id')
data_test = pd.read_csv(r'test.csv', index_col='Id')

In [ ]:
data.head()